In [1]:
import sys
sys.path.append('lib/')

import pandas as pd
import folium
import os
import glob

import TFilter as f
from TDataFrame import TDataFrame

In [2]:
# load data
filepaths = glob.glob(os.path.join("../../../DP/", "*.csv"))

# Loaded TDFs
ltdfs = []

for filename in filepaths:
    df = pd.read_csv(filename, sep=';', encoding='latin-1', low_memory=False)
    
    # add to list
    ltdfs.append(TDataFrame(filename, df))

print(ltdfs)

[<TDataFrame.TDataFrame object at 0x12a5def50>, <TDataFrame.TDataFrame object at 0x12d87ef10>, <TDataFrame.TDataFrame object at 0x12d87cb50>, <TDataFrame.TDataFrame object at 0x12d8920d0>, <TDataFrame.TDataFrame object at 0x12dc27fd0>]


In [3]:
# settings
T_Line = '*'
T_Date = "TODO"

# filter all ltdfs
data = f.filter_tdfs(ltdfs, T_Line)
data

In [6]:
from folium.plugins import HeatMap

# plot into map 
m = folium.Map(prefer_canvas=True)

# plot emb data
for tdf in data:
    print(tdf.line)
    fg = folium.FeatureGroup("Line " + str(tdf.line))
    for c in tdf.emb:
         #folium.Marker(location=[ c[0], c[1] ], fill_color='#43d9de', radius=8).add_to(m)
        folium.CircleMarker(location=[c[0], c[1]],
                            radius=5,
                            color='#FF0000',
                            weight=5,
                            ).add_to(fg)
    fg.add_to(m)
    # HeatMap(tdf.emb).add_to(m)
        
# set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

# layer control
folium.LayerControl().add_to(m)

m.save("map.html")
m

4132.0
1031.0
4191.0
0000
9071.0


In [5]:
# co dalej?
#
# moznost loadnut podla liniek - casove rozmedzie, linka, 
# dalsie data okolo brzdenia
# 